<img style="float: left;padding-right: 10px" width ="40px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/SemanticKernelLogo.png">

## Semantic Kernel - Premortem (What-If) Decision Analysis  

> 📜 "The pre-mortem method has a number of benefits besides identifying trouble spots in a plan – reducing overconfidence and promoting discoveries. The method can strengthen members’ mental models… And it can help create a culture of candor and trust."
>
> -- <cite>Gary Klein (Psychologist, creator of the Pre-mortem method)</cite>  

Decision Intelligence applied in this module:  
* Using Premortem Decision Analysis to improve decision quality & outcomes  
* Using a Premortem decision output for improving of Gathering Intelligence 
* Attain different perspectives using mutliple decision frames  
* Apply a Decision Framework to Premortem Decision Analysis 


### What is a Premortem (What-If Analysis)  

You have probably heard of a **postmortem (post-mortem)**, which is a look back at an actual failure (or success) to understand why it happened and learn from it. In the context of decision making, after the decision outcome is clear you can analyze why the result occured. For example, a hedge fund may host a postmortem meeting after analyzing an unsuccesful move on a tech stock that didn't materialize the return on investment as was planned. 

In a **premortem (pre-mortem)** the outcome is "imagined" and then it is figured out why that potential "imagined" outcome happened. In a premortem, a planning exercise is held where a team imagines that a project or decision has already succeeded or failed. The team then works backward, brainstorming what might have caused things to go wrong. This approach allows everyone to spot potential risks and address them before starting the project. Therefore, it’s a bit like predicting the future and using that prediction to make the forecast stronger. 

### Example Decision Scenario - Purchasing a Car   

Imagine you're about to buy a new car. Let's play a game of "What-If" that the purchase decision results in either two outcomes a: **disaster** or a **successful** outcome. 

#### Premortem Outcome - Disaster  

<img style="display: block; margin: auto;" width ="700px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/Scenarios/Scenario-DecisionPremortem.png">

In a **premortem (pre-mortem)**, you pretend the car purchase turned out to be a disaster. Now, you ask yourself:  
**What went wrong with the decision to buy a new car?**

Here are some potential reasons:  

* **Financial issues:** You realize later you can’t comfortably afford the monthly payments or maintenance costs.  
* **Unexpected repairs:** The car has hidden mechanical problems, leading to costly repairs.  
* **Insurance costs:** Insurance is much higher than expected, impacting your budget.
* **Poor fuel efficiency:** The car uses more gas than you thought, making it expensive to drive.
Resale value drop: The car’s value plummets quickly, leaving you with less if you want to sell or trade it.  

#### Premortem Outcome - Success

In a premortem for a successful car purchase, imagine you bought the car, and it turned out to be a succssful decision. Now, you ask yourself:  
**What went right with the decision to buy a new car?**  

Here are some potential reasons:

* **Affordable and realistic budget:** You set a budget that allowed for comfortable monthly payments and covered insurance, fuel, and maintenance without straining your finances.  
* **Thorough inspection:** You got a detailed inspection before buying, which confirmed the car was in excellent condition and minimized the risk of future repairs.
* **Reliable model choice:** You researched models with a reputation for reliability, so you experience fewer repairs and smooth performance.
* **Fuel efficiency matches needs:** You chose a car that meets your fuel efficiency needs, saving money on gas and fitting well with your daily driving habits.   
* **High resale value:** You picked a make and model that holds its value well, giving you confidence that it’s a good long-term investment.  

#### Premortem (What-If) Decision Analysis 

Imagining successful outcomes helps you focus on actions like sticking to a car budget, getting a full inspection, and choosing a reliable model. You will be more likely to make a purchase you’ll be happy with. By plyaing a "game" of What-If with decision criteria in advance, you could adapt your strategy. For example, you could consider to thoroughly check maintenance records, buy additional warranty, test drive with similar terrain in your area, or reassess your budget to avoid these potential problems.

How can Generative AI help? Using Generative AI, the LLM models can be used to "imagine" premortem outcomes and  brainstorm potential factors why that certain outcome occured. Generative AI agents can act as your "premortem" team. This can lead to additional questions, gathering of intelligence, checklists, negotiation leverage etc. you didn't consider initially. 

### Step 1 - Initialize Configuration Builder & Build the Semantic Kernel Orchestration 

In [1]:
#r "nuget: Microsoft.Extensions.Configuration, 9.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Json, 9.0.0"
#r "nuget: Microsoft.SemanticKernel, 1.45"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.45-alpha"

using Microsoft.Extensions.Configuration.Json;
using Microsoft.Extensions.Configuration;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using System.IO;

var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

// IMPORTANT: You ONLY NEED either Azure OpenAI or OpenAI connectiopn info, not both.
// Azure OpenAI Connection Info using o1 series of models
var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];
// OpenAI Connection Info 
var openAIAPIKey = config["OpenAI:APIKey"];
var openAIModelId = config["OpenAI:ModelId"];

Installed Packages Microsoft.Extensions.Configuration, 9.0.0 Microsoft.Extensions.Configuration.Json, 9.0.0 Microsoft.SemanticKernel, 1.45.0 Microsoft.SemanticKernel.Plugins.Core, 1.45.0-alpha

In [2]:
using Microsoft.Extensions.DependencyInjection.Extensions;
using Microsoft.Extensions.DependencyInjection;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

Kernel semanticKernel;

// Set the flag to use Azure OpenAI or OpenAI. False to use OpenAI, True to use Azure OpenAI
var useAzureOpenAI = true;

// Create a new Semantic Kernel instance
if (useAzureOpenAI)
{
    Console.WriteLine("Using Azure OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddAzureOpenAIChatCompletion(
            deploymentName: azureOpenAIModelDeploymentName,
            endpoint: azureOpenAIEndpoint,
            apiKey: azureOpenAIAPIKey)
        .Build();
}
else
{
    Console.WriteLine("Using OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddOpenAIChatCompletion(
            modelId: openAIModelId,
            apiKey: openAIAPIKey)
        .Build();
}

var chatCompletionService = semanticKernel.Services.GetRequiredService<IChatCompletionService>();

Using Azure OpenAI Service


### Step 2 - Premortem Risk Analysis with AI Reasoning Models

> 📜 "Success is a lousy teacher. It seduces smart people into thinking they can’t lose."
>
> -- <cite>Bill Gates (Co-Founder of Microsoft, Philantropist)</cite>  

In the decision scenario below, an AI model will perform a premortem (What-If) analysis on the decision whether to purchase or lease a car. The AI model will provide risks and items to consider for each of the decision options: purchasing a car or leasing a car. 

In [3]:
// Create a system prompt instruction to override the default system prompt
// Add the System Prompt (Persona) to behave like a decision intelligence assistant
var systemPromptForDecisions = """
You are a Decision Intelligence Assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving.
Apply systems thinking to various scenarios. Provide structured, logical, and comprehensive advice.
""";
var decisionAnalysisPrompt = """
Perform an advanced premortem (What-If) analysis on the following decision scenario:
You are deciding to purchase a new car versus leasing a car.
""";

// How the prompt looks like to the LLM
var simpleDecisionPromptTemplate = $"""
System Prompt: {systemPromptForDecisions}

Request from the user: {decisionAnalysisPrompt}
""";

var simplePromptResponse = await semanticKernel.InvokePromptAsync(decisionAnalysisPrompt);
Console.WriteLine(simplePromptResponse.GetValue<string>());

### Advanced Premortem (What-If) Analysis for Purchasing vs Leasing a Car

A premortem analysis is a structured process to assess potential risks and downsides before committing to a decision. In this exercise, we'll evaluate the possible scenarios under which the decision to purchase or lease a car could fail or lead to undesirable outcomes. By identifying vulnerabilities early, we can take steps to mitigate them or optimize our decision.

---

### Decision Scenario
You are deciding between **purchasing** a new car or **leasing** a car. The goal is to make the most financially sound, practical, and long-term decision. Factors such as cost, flexibility, maintenance, depreciation, lifestyle, and financial goals are being considered.

---

### Premortem: **What Could Go Wrong?**

Below is an analysis of worst-case scenarios for each option.

---

#### **Option 1: Purchasing a Car**
___"What If purchasing the car turns out to be a major mistake?"___

1. **What If You Overestimated Your Bu

The large reasoning model provided a comprehensive set of criteria, risks, mitigation strategy and approaches for the two decisions. This is a great option, but it might be more ideal to provide this in an easy to consume table (checklist) for the user that is easy to compare and contrast. This What-If analysis is easy to consume on a mobile device or even print out and annotate. 

In [4]:
// Create a system prompt instruction to override the default system prompt
// Add the System Prompt (Persona) to behave like a Decision Intelligence assistant
var systemPromptForDecisions = """
You are a Decision Intelligence Assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving.
Apply systems thinking to various scenarios. Provide structured, logical, and comprehensive advice.
""";
var decisionAnalysisPrompt = """
Perform an advanced premortem (What-If) analysis on the following scenario:
You are deciding to purchase a new car versus leasing a car.

Output the risk factors, uncertainties, and potential outcomes ONLY in a Markdown table.
Create a column for each option (Purchase, Lease) and which risk factor or uncertainty applies to each option.
Do not surround the markdown with triple backticks.
""";

// Illustrate how the prompt appears to the LLM
var simpleDecisionPromptTemplate = $"""
System Prompt: {systemPromptForDecisions}

Request from the user: {decisionAnalysisPrompt}
""";

var simplePromptResponse = await semanticKernel.InvokePromptAsync(decisionAnalysisPrompt);
Console.WriteLine(simplePromptResponse.GetValue<string>());

| **Risk Factor/Uncertainty**               | **Purchase**                                     | **Lease**                                       |
|-------------------------------------------|-------------------------------------------------|------------------------------------------------|
| **Depreciation**                          | High upfront cost with potential steep value loss over time. | Risk of overpaying residual value if buyout option is exercised. |
| **Monthly Payments**                      | Higher monthly payments (unless a large down payment is made). | Typically lower monthly payments compared to purchase.           |
| **Mileage Limitations**                   | No restrictions on mileage.                    | Mileage overage fees if limits are exceeded.                      |
| **Maintenance Costs**                     | Out-of-pocket repair costs after the warranty expires.        | Maintenance often covered during the lease term with potential fees for excess we

| **Risk Factor / Uncertainty / Potential Outcome** | **Purchase** | **Lease** |
|---------------------------------------------------|--------------|------------|
| **Depreciation of Vehicle Value**                 | Yes          | No         |
| **Residual Value Risk**                           | Yes          | No         |
| **Potential Negative Equity**                     | Yes          | No         |
| **Mileage Limitations**                           | No           | Yes        |
| **Wear and Tear Charges**                         | No           | Yes        |
| **Early Termination Penalties**                   | No           | Yes        |
| **Upfront Costs**                                 | Higher       | Lower      |
| **Monthly Payments**                              | Generally Higher | Generally Lower |
| **Maintenance Costs**                             | Yes (after warranty expires) | May be included |
| **Ownership at End of Term**                      | Yes          | No         |
| **Flexibility to Upgrade Vehicle**                | Less Flexible | More Flexible |
| **Risk of Obsolescence**                          | Yes          | No         |
| **Total Cost Over Term**                          | Potentially Higher | Potentially Lower |
| **Ability to Build Equity**                       | Yes          | No         |
| **Risk of High Repair Costs**                     | Yes          | Less Risk (covered by warranty) |
| **Resale Value Uncertainty**                      | Yes          | No         |
| **Over-Mileage Charges**                          | No           | Yes        |
| **Ability to Customize Vehicle**                  | Yes          | Restricted |
| **Changes in Personal Circumstances**             | May require selling the car | Early termination fees |
| **Interest Rate Fluctuations**                    | Yes          | Yes        |
| **Tax Implications**                              | Sales tax on full price | Sales tax on monthly payments |
| **Insurance Costs**                               | Standard     | May require higher coverage |
| **Credit Score Impact**                           | Yes          | Yes        |
| **Financial Flexibility**                         | Less         | More       |
| **Opportunity Cost of Capital**                   | Higher (capital tied up) | Lower (more capital available) |
| **End-of-Term Options**                           | Keep or sell the car | Buyout or return the car |

This Markdown table can be further simplified by highlighting which risk factor is more favorable when comparing a car purchase versus a car lease.

| Risk Factor / Uncertainty / Potential Outcome | Purchase | Lease |
|-----------------------------------------------|----------|-------|
| High Upfront Cost                             |          | ☑️    |
| Depreciation of Vehicle Value                 |          | ☑️    |
| Maintenance Costs After Warranty              |          | ☑️    |
| Resale Value Uncertainty                      |          | ☑️    |
| Financing Costs (Interest Rates)              |          |       |
| Changes in Personal Circumstances             | ☑️       |       |
| Technological Obsolescence                    |          | ☑️    |
| Opportunity Cost of Capital                   |          | ☑️    |
| Mileage Limits and Penalties                  | ☑️       |       |
| Wear and Tear Charges                         | ☑️       |       |
| No Equity Built in Vehicle                    | ☑️       |       |
| Early Termination Fees                        | ☑️       |       |
| Insurance Gap Coverage Needed                 | ☑️       |       |
| Flexibility to Modify Car                     | ☑️       |       |
| Total Cost Over Time                          | ☑️       |       |
| Obligation Period                             | ☑️       |       |
| Impact of Market Conditions                   |          | ☑️    |
| Tax Considerations                            |          | ☑️    |
| Credit Score Impact                           |          |       |
| End of Term Options                           |          | ☑️    |
| Flexibility of Use                            | ☑️       |       |


### Step 3 - Premortem Analysis for Radio Telescope Operations 

Exploring space with telescopes is a time consuming investment. Current telescope designs are limited to exploring small areas of space in a certain time frame. Without going into details, the process is much more complex than snapping a picture with a mobile device. Telescopes can be pointed at a specific space area for days, weeks or even more than months! If a telescope is exploring a certain area of space it isn't collecting data on other areas. Scientists and researchers compete for valuable access to telescope resources by providing observatories what space area they want to explore, any cellestial contraints, timeframes, calibration costs, data collection transfer etc. This presents a very interesting Decision Intelligence constraints problem that observatories must manage. 

<img style="display: block; margin: auto;" width ="700px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/Scenarios/Scenario-RadioTelescope.png">

#### Decision Scenario - Premortem for Radio Telescope Operations 

> 📜 "We choose to go to the Moon in this decade and do the other things, not because they are easy, but because they are hard." 
>
> -- <cite>John F. Kennedy (35th President of the United States)</cite>  

A newly commissioned radio telescope stands on the verge of unveiling the proof of alien life, but it is still far from fully calibrated. Despite limited data processing capacity and a software suite that remains untested, an unexpected cosmic event has just begun, hinting at signals that could reveal alien life. Faced with the risk of hardware damage or data corruption from running the telescope prematurely, scientists debate whether to switch it on now or wait until everything is fully operational—knowing they could lose a once-in-a-generation opportunity. Determined not to squander this brief cosmic window, the team conducts a premortem analysis to predict what might go wrong. Armed with these insights, they can better prepare for potential pitfalls and strengthen their chances of catching historic signals from across the galaxy.

Let's assume the scientists decide to rush the radio telescope to operate early to find alien life. Furthermore, assume a that this decision has led to a disaster with hardware failure to the telescope. Even though the (What-If) decision was made to rush the radio telescope operationally, formal scientific telescope operations have been followed. The goal of this premortem (What-If analysis) is to illuminate any risk mitigation strategies and craft a risk mitigation approach that may have been uncovered based on the premortem. 

Techinques like this a very powerful in a variety of business scenarios, where existing operational frameworks exist but simulating an outcome can uncover gaps, emerging risks and other opportunities to optimize. 

In [5]:
// Create a system prompt instruction to override the default system prompt
// Add the System Prompt (Persona) to behave like a decision intelligence assistant
var systemPromptForPremortemRadioTelescope = """
You are a Decision Intelligence Assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving.
Apply systems thinking to various scenarios. Provide structured, logical, and comprehensive advice.
""";
var decisionAnalysisPromptForPremortemRadioTelescope = """
You are a project manager at a new radio telescope facility looking for alien life.  
It is still far from fully calibrated. Despite limited data processing capacity and a software suite that remains untested, 
an unexpected cosmic event has just begun, hinting at signals that could reveal alien life. 
Faced with the risk of hardware damage or data corruption from running the telescope prematurely, 
scientists debate whether to switch it on now or wait until everything is fully operational—knowing 
they could lose a once-in-a-generation opportunity. 

Imagine it is six months from now and the project to detect potential alien signals from the sudden cosmic event has failed. 
The radio telescope was likely activated prematurely, software remained untested, and the team rushed to capture transmissions. 
Despite the urgency, the outcome was disastrous: hardware suffered damage under the increased load, 
data came out corrupted, or no meaningful signals were confirmed. 

Now, step into that moment of failure—what specific issues, oversights, or cascading errors caused this grim scenario?

1) Envision every possible way the radio telescope's early deployment could have led to compromised or unusable data.
2) Identify how incomplete software testing and rushed calibration might have 
contributed to errors in processing or storing signals. 
3) Consider any overlooked safety protocols or hardware precautions that might have led to damage or malfunctions.
4) Examine the roles and responsibilities of the team and think about communication breakdowns, 
rushed decisions, and missed signals or deadlines.
5) Assess how time pressure from the rare cosmic event might have amplified smaller mistakes and 
turned them into systemic failures.

Use the outcomes of this “what if” vision to create a detailed list of preventive measures. 
Then, propose fail-safes, additional resources, or procedural changes to ensure that, 
should you choose to activate the telescope early, you can still capture the event safely and 
effectively—maximizing the chance to detect historic signals without jeopardizing the entire operation.
""";

// How the prompt looks like to the LLM
var fullPromptForPremortemRadioTelescope = $"""
System Prompt: {systemPromptForPremortemRadioTelescope}

Request from the user: {decisionAnalysisPromptForPremortemRadioTelescope}
""";

var simplePromptResponse = await semanticKernel.InvokePromptAsync(fullPromptForPremortemRadioTelescope);
Console.WriteLine(simplePromptResponse.GetValue<string>());

### Retrospective Analysis: The Factors Contributing to Failure

##### 1. **Compromised or Unsable Data**
   - Miscalibrated instrumentation caused erratic signal reception, mixing potential alien signals with noise.
   - Premature data analysis software failed to distinguish genuine cosmic signals from background interference or system artifacts.
   - Corrupted data occurred due to overloaded or improperly configured storage systems.
   - Environmental or system conditions (e.g., thermal imbalances, signal attenuation) were not accounted for, leading to invalid readings.

##### 2. **Impact of Incomplete Software Testing**
   - Algorithms designed to filter or interpret signals were buggy, missing key patterns or mislabeling noise as potential evidence.
   - The signal processing pipeline experienced glitches under real-world load and failed to store or process data in the correct formats.
   - Faulty communication between data streams led to synchronization errors, fragmenting incompl